In [2]:
import sys

import numpy as np

sys.path.insert(0, '..')

In [3]:
biomodel_id = 'BIOMD0000000630'
duration = 100

# sample time series outputs from three simulators
output_copasi = np.array([1.3, 2.4, 4.3, 5.6, 6.7])
output_tellurium = np.array([1.3, 2.4, 4.5, 5.7, 6.8])
output_amici = np.array([1.3, 2.4, 4.6, 5.8, 7.0])
interval_outputs = [output_copasi, output_tellurium, output_amici]

# sample time id for a random interval in the range of duration
time_id = np.random.randint(0, duration)